# J1044 ABUNDANCE SCALES AND METALLICITIES

## Importing Packages

In [66]:
import numpy as np
import pandas as pd

## Metallicity Determination by Hand

In [67]:
logOH12_gal = 7.47
logOH_gal = logOH12_gal - 12

logOH_gal

-4.53

In [68]:
logOH_solar = -3.31

print("J1044 O/H:", logOH_gal)
print("Solar O/H:", logOH_solar)

J1044 O/H: -4.53
Solar O/H: -3.31


To find the metals and grains, we have to find the value of J1044 O/H over solar O/H. However, note that this is a log value and we want to make it linear.

In [69]:
metallicity = (10**logOH_gal) / (10**logOH_solar)

print("J1044 metals and grains:", "{:.2f}".format(metallicity))

J1044 metals and grains: 0.06


## Defining a Metallicity Scale Function

In [70]:
def find_metals(galaxy_OH12):
    solar_logOH = -3.31
    galaxy_logOH = galaxy_OH12 - 12

    galaxy_metals = (10**galaxy_logOH) / (10**solar_logOH)

    return float("{:.2f}".format(galaxy_metals))

In [71]:
def find_metals_with_err(galaxy_OH12, galaxy_OH12_err):
    solar_logOH = -3.31
    galaxy_logOH = galaxy_OH12 - 12
    galaxy_logOH_err = (galaxy_OH12 + galaxy_OH12_err) - 12

    galaxy_metals = (10**galaxy_logOH) / (10**solar_logOH)
    galaxy_metals_1sig = (10**galaxy_logOH_err) / (10**solar_logOH)

    # return float("{.2f,.2f}".format(galaxy_metals,galaxy_metals_1sig))
    return galaxy_metals, galaxy_metals_1sig

In [72]:
if find_metals(7.47) == float("{:.2f}".format(metallicity)):
    print("Metallicity function works!")
    print("Our metals and grains value for J1044 is:", find_metals(7.47))
else:
    print("Not quite right there.")

Metallicity function works!
Our metals and grains value for J1044 is: 0.06


# ELEMENTAL SCALE FUNCTIONS

## Defining a Nitrogen Scale Function

In [73]:
def N_scale(galaxy_logNO):
    solar_logNH = -4.17
    solar_logOH = -3.31
    solar_NO = (10**solar_logNH) * (1 / (10**solar_logOH))

    scale = (10**galaxy_logNO) / (solar_NO)

    return float("{:.2f}".format(scale))

## Defining a Carbon Scale Function

In [74]:
def C_scale(galaxy_logCO):
    solar_logCH = -3.57
    solar_logOH = -3.31
    solar_CO = (10**solar_logCH) * (1 / (10**solar_logOH))

    scale = (10**galaxy_logCO) / (solar_CO)

    return float("{:.2f}".format(scale))

## Defining a Silicon Scale Function

In [75]:
def Si_scale(galaxy_logSiO):
    solar_logSiH = -4.49
    solar_logOH = -3.31
    solar_SiO = (10**solar_logSiH) * (1 / (10**solar_logOH))

    scale = (10**galaxy_logSiO) / (solar_SiO)

    return float("{:.2f}".format(scale))

# CONVERTING GALAXY SPREADSHEET INTO DATAFRAME

In [76]:
galaxy_vals = pd.read_excel('CLASSY_Metallicity_Te.xlsx')

df = galaxy_vals[[
    'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
    'Unnamed: 12'
]]

df1 = df.loc[6:].reset_index(drop=True)
df2 = df1.rename(
    columns={
        'Unnamed: 1': 'Galaxy',
        'Unnamed: 3': 'SDSS_density',
        'Unnamed: 7': 'SDSS_12logOH',
        'Unnamed: 8': 'OH_error (-)',
        'Unnamed: 9': 'OH_error (+)',
        'Unnamed: 12': 'MUSE_12logOH'
    })

galaxies = df2
galaxies.iloc[43, 2] = 7.77

galaxies.head(10)

,Galaxy,SDSS_density,SDSS_12logOH,OH_error (-),OH_error (+),MUSE_12logOH
0,J0021+0052,90,8.28,0.06,0.06,8.14
1,J0036-3333,-,-,-,-,7.47
2,J0127-0619,-,-,-,-,-
3,J0144+0453,-,-,-,-,-
4,J0337-0502,-,-,-,-,-
5,J0405-3648,-,-,-,-,7.22
6,J0808+3948,1250,-,-,-,-
7,J0823+2806,150,8.33,0.05,0.05,-
8,J0926+4427,100,8.05,0.08,0.06,-
9,J0934+5514,-,-,-,-,-


# DETERMINING METALLICITY SCALES FOR ALL GALAXIES

In [77]:
metallicities = []

for SDSS_logOH, MUSE_logOH in zip(galaxies['SDSS_12logOH'],
                                  galaxies['MUSE_12logOH']):
    if SDSS_logOH != '-':
        metal_val = find_metals(SDSS_logOH)
        metallicities.append(metal_val)
    elif MUSE_logOH != '-':
        metal_val = find_metals(MUSE_logOH)
        metallicities.append(metal_val)
    else:
        metallicities.append('-')

galaxies['CLOUDY metals and grains'] = metallicities

# we can now see our updated dataframe with the metallicity values
galaxies.head()

,Galaxy,SDSS_density,SDSS_12logOH,OH_error (-),OH_error (+),MUSE_12logOH,CLOUDY metals and grains
0,J0021+0052,90,8.28,0.06,0.06,8.14,0.39
1,J0036-3333,-,-,-,-,7.47,0.06
2,J0127-0619,-,-,-,-,-,-
3,J0144+0453,-,-,-,-,-,-
4,J0337-0502,-,-,-,-,-,-


# DETERMINING HYDROGEN LOG DENSITY FOR ALL GALAXIES

In [78]:
hdens = []

for density in galaxies['SDSS_density']:
    if (density != '-') and (density != '<100'):
        d_string = str(density)
        power = len(d_string) - 1
        hdens.append(power)
    elif density == '<100':
        hdens.append(1)
    else:
        hdens.append('-')

galaxies['log_density'] = hdens

galaxies.head(10)

,Galaxy,SDSS_density,SDSS_12logOH,OH_error (-),OH_error (+),MUSE_12logOH,CLOUDY metals and grains,log_density
0,J0021+0052,90,8.28,0.06,0.06,8.14,0.39,1
1,J0036-3333,-,-,-,-,7.47,0.06,-
2,J0127-0619,-,-,-,-,-,-,-
3,J0144+0453,-,-,-,-,-,-,-
4,J0337-0502,-,-,-,-,-,-,-
5,J0405-3648,-,-,-,-,7.22,0.03,-
6,J0808+3948,1250,-,-,-,-,-,3
7,J0823+2806,150,8.33,0.05,0.05,-,0.44,2
8,J0926+4427,100,8.05,0.08,0.06,-,0.23,2
9,J0934+5514,-,-,-,-,-,-,-


# SCALING THE ABUNDANCES OF N, C, AND Si

## Reading in Original Elemental Abundances

In [79]:
ele_df = pd.read_csv('metalliciy-N_O_SDSS_MUSE.txt', delimiter=r"\s+")

# add in zero column for Carbon to handle currently missing data
ele_df['CO_SDSS'] = 0.0
ele_df['CO_MUSE'] = 0.0

# add in zero column for Silicon to handle currently missing data
ele_df['Si_SDSS'] = 0.0
ele_df['Si_MUSE'] = 0.0

# add in error columns of placeholder error value 0.1
ele_df['N_err'] = 0.1
ele_df['C_err'] = 0.1
ele_df['Si_err'] = 0.1

ele_df.head()

,Galaxy,ne_SDSS,OH_SDSS,NO_SDSS,ne_MUSE,OH_MUSE,NO_MUSE,CO_SDSS,CO_MUSE,Si_SDSS,Si_MUSE,N_err,C_err,Si_err
0,J0021+0052,90.0,8.28,-1.211556,120.0,8.14,-0.786478,0.0,0.0,0.0,0.0,0.1,0.1,0.1
1,J0036-3333,0.0,0.00,0.000000,120.0,7.47,-1.557300,0.0,0.0,0.0,0.0,0.1,0.1,0.1
2,J0127-0619,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.1,0.1,0.1
3,J0144+0453,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.1,0.1,0.1
4,J0337-0502,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.1,0.1,0.1


## Creating Nitrogen Scale Column

In [80]:
NO_scale_list = []

for obj1, obj2 in zip(ele_df['NO_SDSS'], ele_df['NO_MUSE']):
    if int(obj1) != 0:
        NO_scale_list.append(N_scale(obj1))
    elif int(obj2) != 0:
        NO_scale_list.append(N_scale(obj2))
    else:
        NO_scale_list.append(0.0)

galaxies['NO_scale'] = NO_scale_list

## Creating Carbon Scale Column

In [81]:
CO_scale_list = []

for obj1, obj2 in zip(ele_df['CO_SDSS'], ele_df['CO_MUSE']):
    if int(obj1) != 0:
        CO_scale_list.append(C_scale(obj1))
    elif int(obj2) != 0:
        CO_scale_list.append(C_scale(obj2))
    else:
        CO_scale_list.append(0.0)

galaxies['CO_scale'] = CO_scale_list

## Creating Silicon Scale Column

In [82]:
SiO_scale_list = []

for obj1, obj2 in zip(ele_df['Si_SDSS'], ele_df['Si_MUSE']):
    if int(obj1) != 0:
        SiO_scale_list.append(Si_scale(obj1))
    elif int(obj2) != 0:
        SiO_scale_list.append(Si_scale(obj2))
    else:
        SiO_scale_list.append(0.0)

galaxies['SiO_scale'] = SiO_scale_list

## Updated Galaxies Table With Scale Factors

In [83]:
galaxies.head(15)

,Galaxy,SDSS_density,SDSS_12logOH,OH_error (-),OH_error (+),MUSE_12logOH,CLOUDY metals and grains,log_density,NO_scale,CO_scale,SiO_scale
0,J0021+0052,90,8.28,0.06,0.06,8.14,0.39,1,0.45,0.0,0.0
1,J0036-3333,-,-,-,-,7.47,0.06,-,0.20,0.0,0.0
2,J0127-0619,-,-,-,-,-,-,-,0.00,0.0,0.0
3,J0144+0453,-,-,-,-,-,-,-,0.00,0.0,0.0
4,J0337-0502,-,-,-,-,-,-,-,0.00,0.0,0.0
5,J0405-3648,-,-,-,-,7.22,0.03,-,0.21,0.0,0.0
6,J0808+3948,1250,-,-,-,-,-,3,0.00,0.0,0.0
7,J0823+2806,150,8.33,0.05,0.05,-,0.44,2,0.59,0.0,0.0
8,J0926+4427,100,8.05,0.08,0.06,-,0.23,2,0.38,0.0,0.0
9,J0934+5514,-,-,-,-,-,-,-,0.00,0.0,0.0


add in other columns

# CREATING THE FINAL TABLE OF PARAMETERS

## Building the List of Galaxies and Their Indices

In [86]:
# creating a list of galaxy names
gal_grid_list = []

# creating a list of the indices
param_index = []

# number of original galaxies (45)
num_galaxies = len(galaxies)

# number of grids per parameter
n = 7

# empty lists of parameters to match indices
imet = []
iLu = []
idens = []
iNit = []
iCarb = []
iSic = []

# LOOP THROUGH EACH GALAXY:
for i in range(num_galaxies):

    # LOOP THROUGH METALLICITY:
    for x_1 in range(1,n+1):
        
        # LOOP THROUGH LOG U:
        for x_2 in range(1,n+1):
            
            # LOOP THROUGH DENSITY:
            for x_3 in range(1,n+1):
                
                # LOOP THROUGH CNSi:
                for x_4 in range(1,n+1):
                    
                    ##### DEFINE THE METALLICITY SOLAR SCALE
                    if (galaxies.iloc[i,2] != '-') and (galaxies.iloc[i,4] != '-'):
                        # define the original OH value and error value
                        og_OHval = galaxies.iloc[i,2]
                        err_OHval = galaxies.iloc[i,4]
                        
                        begin_logval = og_OHval - (3 * err_OHval)
                        final_val = find_metals(begin_logval + err_OHval*(x_1 - 1))
                        imet.append(final_val)
                    # if OH abundance ratio is null (make sure this doesn't mess up index)
                    else:
                        imet.append(0.0)
                        
                    ##### DEFINE THE LOG_U PARAMETER STARRRR
                    logU = -4
                    step_logU = 0.5
                    init_logU = logU - (3*step_logU)
                    final_logU = init_logU + step_logU*(x_2 - 1)
                    
                    iLu.append(final_logU)
                    
                    ##### DEFINE THE DENSITY PARAMETER
                    idens.append(2)
                    
                    ##### DEFINE ELEMENTAL SCALES
                    ### NITROGEN
                    if (galaxies.iloc[i,8] != 0.00):
                        # setting the error value
                        err1_NOval = ele_df.iloc[i,11]
                        
                        # if-else to set the original NO value
                        if (int(ele_df.iloc[i,3]) != 0):
                            # NO SDSS
                            og1_NOval = ele_df.iloc[i,3]
                        else:
                            # NO MUSE
                            og1_NOval = ele_df.iloc[i,6]
                        
                        begin1_NOval = og1_NOval - (3*err1_NOval)
                        final1_NOval = N_scale(begin1_NOval + err1_NOval*(x_4 - 1))
                        iNit.append(final1_NOval)
                        
                    # checking that C isn't null so that we can scale N with C
                    elif (galaxies.iloc[i,9] != 0.0):
                        # setting the error value
                        err1_COval = ele_df.iloc[i,12]
                        
                        # if-else to set the original NO value
                        if (int(ele_df.iloc[i,7]) != 0):
                            # CO SDSS
                            og1_COval = ele_df.iloc[i,7]
                        else:
                            # CO MUSE
                            og1_COval = ele_df.iloc[i,8]
                        
                        begin1_COval = og1_COval - (3*err1_COval)
                        final1_COval = C_scale(begin1_COval + err1_COval*(x_4 - 1))
                        iNit.append(final1_COval)
                    
                    # appending zero if both Nitrogen and Carbon are null
                    else:
                        iNit.append(0.0)

                    ### CARBON
                    if (galaxies.iloc[i,9] != 0.0):
                        # setting the error value
                        err2_COval = ele_df.iloc[i,12]
                        
                        # if-else to set the original NO value
                        if (int(ele_df.iloc[i,7]) != 0):
                            # CO SDSS
                            og2_COval = ele_df.iloc[i,7]
                        else:
                            # CO MUSE
                            og2_COval = ele_df.iloc[i,8]
                        
                        begin2_COval = og2_COval - (3*err2_COval)
                        final2_COval = C_scale(begin2_COval + err2_COval*(x_4 - 1))
                        iCarb.append(final2_COval)
                    
                    # if N is not null, scale C with N
                    elif (galaxies.iloc[i,8] != 0.00):
                        # setting the error value
                        err2_NOval = ele_df.iloc[i,11]
                        
                        # if-else to set the original NO value
                        if (int(ele_df.iloc[i,3]) != 0):
                            # NO SDSS
                            og2_NOval = ele_df.iloc[i,3]
                        else:
                            # NO MUSE
                            og2_NOval = ele_df.iloc[i,6]
                        
                        begin2_NOval = og2_NOval - (3*err2_NOval)
                        final2_NOval = N_scale(begin2_NOval + err2_NOval*(x_4 - 1))
                        iCarb.append(final2_NOval)
                    
                    # append zero if both Carbon and Nitrogen are null
                    else:
                        iCarb.append(0.0)

                    ### SILICON
                    if (galaxies.iloc[i,10] != 0.0):
                        # setting the error value
                        err_Sival = ele_df.iloc[i,13]
                        
                        # if-else to set the original SiO value
                        if (int(ele_df.iloc[i,9]) != 0):
                            # SiO SDSS
                            og_SiOval = ele_df.iloc[i,9]
                        else:
                            # SiO MUSE
                            og_SiOval = ele_df.iloc[i,10]
                        
                        begin_SiOval = og_SiOval - (3*err_SiOval)
                        final_SiOval = C_scale(begin_SiOval + err_SiOval*(x_4 - 1))
                        iSic.append(final_SiOval)
                    
                    # append one if Silicon is null
                    else:
                        iSic.append(1.0)

                    # define the galaxies column
                    gal_grid_list.append(galaxies.iloc[i,0])
                    
                    # define the index
                    param_index.append(int(str(x_1) + str(x_2) + str(x_3) + str(x_4)))

# add all the lists to a table
param_df = pd.DataFrame()

param_df['Index'] = param_index
param_df['Galaxy'] = gal_grid_list
param_df['OH scaled'] = imet
param_df['LogU'] = iLu
param_df['Hydrogen Density'] = idens
param_df['N scale'] = iNit
param_df['C scale'] = iCarb
param_df['Si scale'] = iSic

## Understanding Our Parameter DataFrame

In [87]:
param_df.head()

,Index,Galaxy,OH scaled,LogU,Hydrogen Density,N scale,C scale,Si scale
0,1111,J0021+0052,0.26,-5.5,2,0.22,0.22,0.0
1,1112,J0021+0052,0.26,-5.5,2,0.28,0.28,0.0
2,1113,J0021+0052,0.26,-5.5,2,0.35,0.35,0.0
3,1114,J0021+0052,0.26,-5.5,2,0.45,0.45,0.0
4,1115,J0021+0052,0.26,-5.5,2,0.56,0.56,0.0


So given the way the loop is set up, we step through our grid initally from the last parameters, namely the elemental scales. We can see from the few values displayed above that this galaxy, J0021+0052, had a null value for either it's NO abundance or CO abundance but has the other value. Therefore, we scaled the missing value with the non-missing one and should expect that every iteration of this galaxy have equal scale factors for N and C.

In [89]:
print('Number of total galaxies (input files):', len(param_df))

Number of total galaxies (input files): 108045


In [92]:
print('Number of variations per galaxy (including original):', int(len(param_df) / 45))

Number of variations per galaxy (including original): 2401


## Checking that the Values Match the Indices

check by cross-referencing the values of parameters with the same single-value index but different across the general index

In [ ]:
errors = []

# check that the indices match for the first five algorithms
for i in range(1,6):
    # defined as galaxy i in n where (n=45)
    index = i*2401
    
    for j in range(1,8):
        index_selection = [int(x) for x in str(param_df.iloc[index])]

Could also try using data.loc 'where' method to access all iterations where the galaxy and the and the parameter variation are the same. Make this a list, then analyze the list to see if there are any differing values, which we can classify as errors. The expectation is that there is either exist some n number of errors or zero errors. Given n, we can use this to deduce where the errors are being raised in the index, since we have a set of loops with differing ranges that are related to our value n.

DO NOT SKIP THIS PART

## Creating a Dictionary to More Easily Access Individual Galaxies

In [94]:
# initialize an empty dictionary
pf_collection = {}

# iterate through original 45 galaxies
for gal in range(45):
    first_index = 2401 * gal
    last_index = 2401 * (gal + 1) - 1
    temp_df = param_df.iloc[first_index:last_index]

    # our key is the name of the galaxy
    key = str(galaxies.iloc[gal, 0])

    pf_collection[key] = temp_df

# display our new dictionary
pf_collection

{'J0021+0052':       Index      Galaxy  OH scaled  LogU  Hydrogen Density  N scale  C scale  \
 0      1111  J0021+0052       0.26  -5.5                 2     0.22     0.22   
 1      1112  J0021+0052       0.26  -5.5                 2     0.28     0.28   
 2      1113  J0021+0052       0.26  -5.5                 2     0.35     0.35   
 3      1114  J0021+0052       0.26  -5.5                 2     0.45     0.45   
 4      1115  J0021+0052       0.26  -5.5                 2     0.56     0.56   
 ...     ...         ...        ...   ...               ...      ...      ...   
 2395   7772  J0021+0052       0.59  -2.5                 2     0.28     0.28   
 2396   7773  J0021+0052       0.59  -2.5                 2     0.35     0.35   
 2397   7774  J0021+0052       0.59  -2.5                 2     0.45     0.45   
 2398   7775  J0021+0052       0.59  -2.5                 2     0.56     0.56   
 2399   7776  J0021+0052       0.59  -2.5                 2     0.71     0.71   
 
       Si sc

## A Guide to the Galaxy Index

In case the index is not clear, there are four numbers in the index with each number being associated with a different parameter that contains a grid: 

1st integer ----> <span class="mark">Metallicity</span>

2nd integer ----> <span class="mark">Nitrogen (N) scale factor</span>

3rd integer ----> <span class="mark">Carbon (C) scale factor</span>

4th integer ----> <span class="mark">Silicon (Si) scale factor</span>

Each integer ranges from 1-7 because we are taking the uncertainty of the previosuly found values for the original galaxies, and we are multiplying this uncertainty by 3 in both the (+) and (-) direction.

Therefore, a value of 6.4 with +/- uncertainty of 0.2 will give us a range of (5.8,7.0) with a step factor of 0.2. Thus, we will have galaxies with values for  [5.8, 6.0, 6.2, 6.4, 6.6, 6.8, 7.0] with the corresponding index  [1, 2, 3, 4, 5, 6, 7]

Given this, we can note that the original value for a galaxy will be marked with the index 4 for that parameter and the galaxy with all the original parameter values is that with index 4444.

# SAVING ALL THE DATA TO A FITS FILE

In [ ]:
from astropy.io import fits as pyfits
path_to_outfiles = '...'
outfile = path_to_outfiles + 'classy_cloudy_grid_info.fits'
hdr = pyfits.Header()
head = pyfits.PrimaryHDU(header=hdr)
hdu  = pyfits.HDUList([head])
for i in range(len(galaxies)):
# Define columns ...
  	 name = galaxies.iloc[i,0]
	 GAL = pyfits.Column(name='GALAXY', array=np.array(pf_collection[name]['galaxies']), format='char')
	 IND = pyfits.Column(name='INDEX', array=np.array(pf_collection[name]['index']), format='I')
	 MET = pyfits.Column(name='METALICITY', array=np.array(pf_collection[name]['metallicity']), format='D', unit='Percent Z_sol')
	....
        hdu.append(pyfits.BinTableHDU.from_columns([GAL, IND, MET, ... ]))
        hdu[i].header['EXTNAME'] = name
hdu.writeto(outfile,overwrite=True)
hdu.close()

# CREATING INPUT FILES FOR ALL GALAXY VARIATIONS

In [32]:
def input_files(file, galaxy_txt, ion_param, ion_lb, ion_ub, ion_step, hden, metal, mg_lb, mg_ub, mg_step, n_scale, c_scale, si_scale):
    print('###############')
    print('title', file)
    print('###############')
    print('set save prefix "' + file + '"')
    print('table SED', galaxy_txt)
    print('ionization parameter', ion_param,'vary')
    print('Grid', ion_lb, 'to', ion_ub,'step', ion_step)
    print('hden', hden)
    print('sphere')
    print('abundances gass10 no grains')
    print('Orion grains')
    print('metals and grains', metal, 'vary')
    print('Grid', mg_lb, 'to', mg_ub, 'step', mg_step, 'linear')
    print('element scale factor nitrogen', n_scale)
    print('element scale factor carbon', c_scale)
    print('element scale factor silicon', si_scale)
    print('iterate to convergence')

input_files('file', 'J10.txt', -2,-3,-1,.25,2,0.05,0,1,0.25,0.25,0.25,0.1)

###############
title file
###############
set save prefix "file"
table SED J10.txt
ionization parameter -2 vary
Grid -3 to -1 step 0.25
hden 2
sphere
abundances gass10 no grains
Orion grains
metals and grains 0.05 vary
Grid 0 to 1 step 0.25 linear
element scale factor nitrogen 0.25
element scale factor carbon 0.25
element scale factor silicon 0.1
iterate to convergence
